In [2]:
def min_max(arr):
    comparisons = 0

    if len(arr) % 2 == 0:
        comparisons += 1
        if arr[0] < arr[1]:
            min_val, max_val = arr[0], arr[1]
        else:
            min_val, max_val = arr[1], arr[0]
        start = 2
    else:
        min_val = max_val = arr[0]
        start = 1

    for i in range(start, len(arr), 2):
        comparisons += 1
        if arr[i] < arr[i + 1]:
            small, large = arr[i], arr[i + 1]
        else:
            small, large = arr[i + 1], arr[i]

        comparisons += 1
        if small < min_val:
            min_val = small

        comparisons += 1
        if large > max_val:
            max_val = large

    return min_val, max_val, comparisons

In [3]:
import random

"""
re-used from quicksort
"""
def partition(a, p, r):
    x = a[r]
    i = p - 1
    for j in range(p, r):
        if a[j] <= x:
            i += 1
            a[i], a[j] = a[j], a[i]
    a[i + 1], a[r] = a[r], a[i + 1]
    return i + 1
    
"""
re-used from quicksort
"""
def randomized_partition(a, p, r):
    i = random.randint(p, r)
    a[r], a[i] = a[i], a[r]
    return partition(a, p, r)

In [5]:
"""
i = 0 | smallest
i = len(a) // 2 | median
i = len(a) - 1 | largest
"""
def randomized_select(a, p, r, i):
    if p == r:
        return a[p]
    q = randomized_partition(a, p, r)
    k = q - p
    if i == k:
        return a[q]
    elif i < k:
        return randomized_select(a, p, q - 1, i)
    else:
        return randomized_select(a, q + 1, r, i - k - 1)

In [6]:
"""
iterative version
"""
def randomized_select_iterative(a, i):
    p = 0
    r = len(a) - 1
    while True:
        if p == r:
            return a[p]

        q = randomized_partition(a, p, r)
        k = q - p
        if i == k:
            return a[q]
        elif i < k:
            r = q - 1
        else:
            i = i - k - 1
            p = q + 1

In [7]:
def partition_around(a, p, r, x):
    for i in range(p, r + 1):
        if a[i] == x:
            a[i], a[r] = a[r], a[i]
            break

    pivot = a[r]
    i = p - 1

    for j in range(p, r):
        if a[j] <= pivot:
            i += 1
            a[i], a[j] = a[j], a[i]

    a[i + 1], a[r] = a[r], a[i + 1]
    return i + 1

In [9]:
"""
Pivot is not random
Pivot is guaranteed to be "good enough"

At least:
3n/10 elements are discarded every level

Worst-case O(n)
"""

def select(arr, k):
    if len(arr) == 1:
        return arr[0]

    groups = [arr[i:i+5] for i in range(0, len(arr), 5)]

    medians = []
    for group in groups:
        group.sort()
        medians.append(group[len(group)//2])

    pivot = select(medians, (len(medians) + 1)//2)

    lows = [x for x in arr if x < pivot]
    highs = [x for x in arr if x > pivot]
    pivots = [x for x in arr if x == pivot]

    if k <= len(lows):
        return select(lows, k)
    elif k <= len(lows) + len(pivots):
        return pivot
    else:
        return select(highs, k - len(lows) - len(pivots))